# Image Classifier
_Ucadity Intro to Deep Learning Project_

In [ ]:
# Imports here
import torch
from torchvision import datasets, transforms, models
from torch import nn, optim
import torch.nn.functional as F
!pip install helper
import helper
import json
from workspace_utils import active_session
from collections import OrderedDict

In [ ]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
tune_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [ ]:
# Define the transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(p=0.25),
                                      transforms.RandomVerticalFlip(p=0.25),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
tune_data = datasets.ImageFolder(tune_dir, transform=test_transforms)
test_data = datasets.ImageFolder(test_dir, transform=test_transforms)

# Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=30, shuffle=True)
tuneloader = torch.utils.data.DataLoader(tune_data, batch_size=30, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=30, shuffle=True)

In [ ]:
# label for classification
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [ ]:
# -- BUILD THE MODEL
# Run on GPU if possible
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') 

# Load the pre-built and pre-trained model
model = models.vgg16(pretrained=True)

# Freeze the parameters for the pre-trained model, so we dont backprop through them
for param in model.parameters():
    param.requires_grad = False
    
# Define model
from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(25088, 2500)),
    ('relu1', nn.ReLU()),
    ('do1', nn.Dropout(0.2)),
    ('fc2', nn.Linear(2500, 102)),
    ('output', nn.LogSoftmax(dim=1))
]))

# Define the model to be trained
model.classifier = classifier # the model pre-trained model VGG doesnt have .fc but .classifier

# Define loss function
criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.003)
 
# Send model to device
model.to(device)

In [ ]:
# -- TRAIN THE MODEL

# For long running code keep the session active
with active_session():
    
    # Def of variables
    epochs = 1 # nr of trainings
    steps = 0
    running_loss = 0
    print_every = 5
    
    # Training
    for epoch in range(epochs): # nr of traing times
        
        for inputs, labels in trainloader: # loop through the data
            
            steps += 1 # train steps
            
            # Move input and label tensors to the default device (GPU if available)
            inputs, labels = inputs.to(device), labels.to(device)
            
            # Zero the gradients - very important
            optimizer.zero_grad()
            
            # Forward pass -> log probabilities
            logps = model.forward(inputs)
            
            # Define loss
            loss = criterion(logps, labels)
            
            # Backward pass
            loss.backward()
            
            # Take a step towards lower loss
            optimizer.step()
            
            # Keep track on the total loss
            running_loss += loss.item()
            
            # Test the model every print_every time
            if steps % print_every == 0:
                test_loss = 0
                accuracy = 0
                model.eval() # set model in evaluation mode
                
                with torch.no_grad(): # reduces memory usage - we dont need to calculate the gradients in evaluation mode
                    
                    for images, labels in testloader:
                        logps = model(images) # log of probability
                        loss = criterion(logps, labels)
                        test_loss += loss.item()

                        # calculate the accuracy
                        ps = torch.exp(logps) # get the actual probability
                        top_p, top_class = ps.topk(5, dim=1) # top probabilities and classes
                        equals = top_class == labels.view(*top_class.shape) # check how many images which have the correct classification
                        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
                        # after getting the top_class of the top_probability from torch.exp(output), we equal the top_class with the images labels (targets) to check if they match or not. The result from this equality is binary values [0,1]. Does this mean that 1 refers to class-label matching and 0 refers to class-label mismatching? Yes

                print(f"Epoch {epoch+1}/{epochs}.. "
                      f"Train loss: {running_loss/print_every:.3f}.. "
                      f"Test loss: {test_loss/len(testloader):.3f}.. "
                      f"Test accuracy: {accuracy/len(testloader):.3f}")
            
                running_loss = 0
                model.train()

